In [1]:
import pandas as pd
import rdflib
from rdflib.namespace import RDF
from rdflib import Namespace
from rdflib import URIRef, Literal

In [2]:
#!pip install rdflib

In [3]:
#Graph declaration
g=rdflib.Graph()

In [4]:
#************Change Path to files************
#main_class=pd.read_csv("C:/Users/SESA456791/Desktop/Ontology-Bh/main_class.csv") 
meta_list=pd.read_csv("C:/Users/SESA456791/Desktop/Ontology-Bh/Files/metaFullList.csv", usecols=["org_type","org"])

In [5]:
#Namespace declarations
org=Namespace("http://www.w3.org/ns/org#")
skos=Namespace("http://www.w3.org/2004/02/skos/core#")
bhorg=Namespace("http://semanticwebindia.in/bharathi/ont/org/")
bhorgres="http://semanticwebindia.in/bharathi/res/org/"
bhcatres=Namespace("http://semanticwebindia.in/bharathi/res/cat/")

In [6]:
#Generation of Concept Scheme and Top Concept
##Creating Concept Scheme as OrganisationType
g.add((URIRef(bhcatres+'OrganisationType'),RDF.type, skos.ConceptScheme))
##Creating topOT as top-concept under conceptScheme:OrganisationType
g.add((URIRef(bhcatres+'topOT'), RDF.type, skos.Concept ))
g.add((URIRef(bhcatres+'topOT'), skos.prefLabel, Literal('Top Concept for Organisation Type')))
g.add((URIRef(bhcatres+'topOT'), skos.altLabel, Literal('Top Concept for Organisation Type')))
g.add((URIRef(bhcatres+'topOT'), skos.inScheme,URIRef(bhcatres+'OrganisationType')))
g.add((URIRef(bhcatres+'OrganisationType'), skos.hasTopConcept,URIRef(bhcatres+'topOT')))

In [7]:
#Creating skos(Government) as broader for topOT
n_cat=1
concept_id=URIRef(bhcatres+str('2')+str(n_cat))
g.add((concept_id,RDF.type,skos.Concept))
g.add((concept_id, skos.prefLabel, Literal("Government")))
g.add((concept_id, skos.altLabel, Literal("Government")))
g.add((concept_id, skos.broader, bhcatres.topOT))
g.add((bhcatres.topGOT, skos.broaderOf, concept_id))
g.add((concept_id,skos.inScheme,URIRef(bhcatres+'OrganisationType')))

## A-Box

In [8]:
bhcatres="http://semanticwebindia.in/bharathi/res/cat/"
bhorgres="http://semanticwebindia.in/bharathi/res/org/"
bhorg="http://semanticwebindia.in/bharathi/ont/org/"

#*****************Change path to file metaFullList.csv *****************
#meta_list=pd.read_csv("C:/Users/SESA456791/Desktop/Ontology-Bh/metaFullList.csv", usecols=["org_type","org"])
#print(meta_list.head())
meta_list=meta_list[(meta_list.org_type).str.lower()!="city"]
meta_list.dropna(inplace=True)
meta_list.reset_index(inplace=True)

In [9]:
#Creating triples for A Box
global n_org
n_org=1

org=Namespace("http://www.w3.org/ns/org#")
bhorg=Namespace("http://semanticwebindia.in/bharathi/ont/org/")
bhcatres=Namespace("http://semanticwebindia.in/bharathi/res/cat/")
bhorgres="http://semanticwebindia.in/bharathi/res/org/"
skos=Namespace("http://www.w3.org/2004/02/skos/core#")

#for i in meta_list.shape(0):
rng=meta_list.shape[0]
for i in range(rng): 
    ls=meta_list['org'][i].split(',') #list of all organisations one cell of org_1
    for j in range(len(ls)):
        #Scrubbing the data clean
        org_str_raw=ls[j].replace("'","")
        org_str_raw=org_str_raw.replace("[","")
        org_str_raw=org_str_raw.replace("]","")
        org_str_raw=org_str_raw.strip()
        org_str=Literal(org_str_raw) 
        if (None,None, org_str) not in g:
            n_org+=1
            n_org_uri=URIRef(bhorgres+str('1')+str(n_org))
            g.add((n_org_uri, RDF.type, bhorg.Organisation))
            g.add((n_org_uri, org.identifier, Literal(n_org)))
            g.add((n_org_uri, bhorg.hasName, org_str))
            g.add((n_org_uri, bhorg.hasAlternateName, org_str))
            #add government as organisation type
            g.add((n_org_uri, org.hasOrganizationType, URIRef(bhcatres+str('2')+str(n_cat))))

In [10]:
g.serialize(format='nt', destination='C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Outputs/Graph_metafile.nt')

In [11]:
print(len(g))

582
